In [1]:
#import dependencies

import json
from pprint import pprint
import numpy as np
import pandas as pd
import os
import csv

from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, Float

import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Boolean

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
# for importing the 1000 row kaggle dataset
# url = 'https://raw.githubusercontent.com/ajdeschenes/FinalProject/master/IMDB-Movie-Data.csv'
# imdbData = pd.read_csv(url)


In [3]:
# GOOGLE COLAB CELL

# for importing large ratings dataset direct from imdb 
# RUN THIS CELL THE FIRST TIME YOU IMPORT DATA                                             


# from google.colab import files
# uploaded = files.upload()

In [4]:
# GOOGLE COLAB CELL

# RUN THIS CELL THE FIRST TIME YOU IMPORT DATA                                             

# import io
# imdbRatings = pd.read_csv(io.BytesIO(uploaded['title.ratings.csv']))


In [5]:
# RUN THIS CELL IF DATA IS ALREADY LOADED IN FILES

imdbRatings = pd.read_csv('../title.ratings.csv')

In [6]:
# GOOGLE COLAB CELL

# for importing large ratings dataset direct from imdb 
# RUN THIS CELL THE FIRST TIME YOU IMPORT DATA                                             


# from google.colab import files
# uploaded = files.upload()

In [7]:
# GOOGLE COLAB CELL

# RUN THIS CELL THE FIRST TIME YOU IMPORT DATA                                             

# import io
# imdbData = pd.read_csv(io.BytesIO(uploaded['title.basics.csv']))

In [8]:
# RUN THIS CELL IF DATA IS ALREADY LOADED IN FILES

imdbData = pd.read_csv('../title.basics.csv')

/anaconda3/envs/MachineLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
imdbData = imdbData.dropna()
imdbRatings = imdbRatings.dropna()

In [10]:
mergedImdb = imdbRatings.merge(imdbData, left_on='tconst', right_on = 'tconst')


In [11]:
mergedImdb = mergedImdb[(mergedImdb.titleType == 'movie')]


In [12]:
mergedImdb.head()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,5.5,80,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
144,tt0000147,5.2,301,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
249,tt0000335,6.3,39,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama"
357,tt0000574,6.2,516,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama"
375,tt0000615,4.8,15,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama


In [13]:
mergedImdb = mergedImdb.drop(columns = ['numVotes', 'titleType', 'originalTitle', 'endYear'])

In [14]:
mergedImdb = mergedImdb[(mergedImdb.runtimeMinutes != '\\N') & (mergedImdb.averageRating != '\\N') & (mergedImdb.isAdult != '\\N') & (mergedImdb.startYear != '\\N') & (mergedImdb.runtimeMinutes != '\\N') & (mergedImdb.genres != '\\N')]

/anaconda3/envs/MachineLearning/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [15]:
mergedImdb.dtypes

tconst             object
averageRating     float64
primaryTitle       object
isAdult             int64
startYear          object
runtimeMinutes     object
genres             object
dtype: object

In [16]:
mergedImdb[['Genre1','Genre2', 'Genre3']] = mergedImdb.genres.str.split(",",expand=True) 


In [17]:
mergedImdb = mergedImdb.drop(columns = 'genres')

In [18]:
# convert None to empty Strings
mask = mergedImdb.applymap(lambda x: x is None)
cols = mergedImdb.columns[(mask).any()]
for col in mergedImdb[cols]:
    mergedImdb.loc[mask[col], col] = ''
mergedImdb.head()

,tconst,averageRating,primaryTitle,isAdult,startYear,runtimeMinutes,Genre1,Genre2,Genre3
8,tt0000009,5.5,Miss Jerry,0,1894,45,Romance,,
144,tt0000147,5.2,The Corbett-Fitzsimmons Fight,0,1897,20,Documentary,News,Sport
357,tt0000574,6.2,The Story of the Kelly Gang,0,1906,70,Biography,Crime,Drama
411,tt0000679,4.8,The Fairylogue and Radio-Plays,0,1908,120,Adventure,Fantasy,
671,tt0001184,3.1,Don Juan de Serrallonga,0,1910,58,Adventure,Drama,


In [19]:
# CLEAN KAGGLE DATASET

# imdbData = imdbData.drop(columns = ['Description','Actors','Votes','Metascore'])
# imdbData["Genre"]= imdbData["Genre"].str.split(",", n = 1, expand = True) 
# imdbData.head()
# modelData = imdbData.drop( columns = ['Title', 'Rank'])
# modelData.head()

In [20]:
# APPLY NEURAL NETWORK TO KAGGLE DATASET

# from sklearn.preprocessing import LabelEncoder

# # Step 1: Label-encode genre
# label_encoder = LabelEncoder()
# label_encoder.fit(modelData['Genre'])
# modelData["Genre"] = label_encoder.transform(modelData['Genre'])

# # Step 1: Label-encode Director
# label_encoder = LabelEncoder()
# label_encoder.fit(modelData['Director'])
# modelData["Director"] = label_encoder.transform(modelData['Director'])

# y = modelData.Rating
# X = modelData.drop(columns = "Rating")

# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# from sklearn.preprocessing import StandardScaler

# X_scaler = StandardScaler().fit(X_train)

# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

# from tensorflow.keras.models import Sequential

# model = Sequential()

# from tensorflow.keras.layers import Dense
# number_inputs = 5
# number_hidden_nodes = 4
# model.add(Dense(units=number_hidden_nodes,
#                 activation='relu', input_dim=number_inputs))

# model.summary()

# #number_classes = 2
# model.add(Dense(units=6, activation='relu', input_dim=2))
# model.add(Dense(units=2, activation='softmax'))

# model.compile(optimizer='adam',
#               loss='mean_absolute_error',
#               metrics=['mean_absolute_error'])

# model.fit(
#     X_train_scaled,
#     y_train,
#     epochs=1000,
#     shuffle=True,
#     verbose=5
# )

# model_loss, model_accuracy = model.evaluate(
#     X_test_scaled, y_test, verbose=5)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# import numpy as np
# new_data = np.array([[2, 0, 6, 101, 200]])
# print(f"Predicted Rating: {model.predict(new_data)}")

In [21]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode genre columns

# count = 1

# while (count <4):
  
#   label_encoder.fit(str(mergedImdb['Genre1']))
#   mergedImdb['Genre1'] = label_encoder.transform(mergedImdb['Genre1'])


label_encoder1 = LabelEncoder()
label_encoder1.fit(mergedImdb['Genre1'])
mergedImdb['Genre1'] = label_encoder1.transform(mergedImdb['Genre1'])

label_encoder2 = LabelEncoder()
label_encoder2.fit(mergedImdb['Genre2'])
mergedImdb['Genre2'] = label_encoder2.transform(mergedImdb['Genre2'])

label_encoder3 = LabelEncoder()
label_encoder3.fit(mergedImdb['Genre3'])
mergedImdb['Genre3'] = label_encoder3.transform(mergedImdb['Genre3'])

    

# # Step 1: Label-encode Director
# label_encoder = LabelEncoder()
# label_encoder.fit(modelData['Director'])
# mergedImdb["Director"] = label_encoder.transform(mergedImdb['Director'])



In [28]:
mergedImdb.head()

,tconst,averageRating,primaryTitle,isAdult,startYear,runtimeMinutes,Genre1,Genre2,Genre3
8,tt0000009,5.5,Miss Jerry,0,1894,45,18,0,0
144,tt0000147,5.2,The Corbett-Fitzsimmons Fight,0,1897,20,7,17,20
357,tt0000574,6.2,The Story of the Kelly Gang,0,1906,70,4,6,7
411,tt0000679,4.8,The Fairylogue and Radio-Plays,0,1908,120,2,10,0
671,tt0001184,3.1,Don Juan de Serrallonga,0,1910,58,2,8,0


In [23]:
# load imdb data into MySQL database

In [29]:
conn_str = "root:helpme01@localhost/imdbData?charset=utf8"
engine = create_engine(f'mysql://{conn_str}')

if not database_exists(engine.url):
    create_database(engine.url)
    
engine.table_names()

[]

In [30]:
metadata = MetaData(engine)

movies = Table('movies', metadata,
                 Column('tconst', String(50), primary_key=True),
                 Column('primaryTitle', String(500)),
                 Column('averageRating', Float),
                 Column('isAdult', Boolean),
                 Column('startYear', Integer),
                 Column('runtimeMinutes', Integer),
                 Column('Genre1', Integer),
                 Column('Genre2', Integer),
                 Column('Genre3', Integer))

metadata.create_all()
for _t in metadata.tables:
    print("Table: ",_t)

Table:  movies


In [31]:
mergedImdb.to_sql(name="movies", con=engine, if_exists="append",index=False)

In [31]:
# Teach and save DecisionTreeRegressor model

In [32]:
y = mergedImdb.averageRating
X = mergedImdb.drop(columns = ['averageRating', 'tconst'])

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [34]:
# from sklearn import tree
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X_train, y_train)
# clf.score(X_test, y_test)

In [35]:
# 
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.07043639256222312

In [36]:

testVals = [0, 2017, 120, 2, 0, 0]
rf.predict([testVals])

array([6.28427952])

In [37]:
import pickle
with open('../imdbModel', 'wb') as f:
    pickle.dump(rf, f)